In [1]:
import polars as pl
import json
from datetime import datetime, timedelta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
import xgboost as xgb
from pathlib import Path

In [2]:
pl.Config.set_tbl_rows(50)
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [3]:
# File paths - adjust these to your actual paths
DATA_DIR = Path("./data")  # Change to your data directory
SALES_FILE = DATA_DIR / "valio_aimo_sales_and_deliveries_junction_2025.csv"
PURCHASES_FILE = DATA_DIR / "valio_aimo_purchases_junction_2025.csv"
PRODUCT_FILE = DATA_DIR / "valio_aimo_product_data_junction_2025.json"
REPLACEMENT_FILE = DATA_DIR / "valio_aimo_replacement_orders_junction_2025.csv"

In [4]:
df_sales = pl.read_csv(SALES_FILE)
print(f"Sales & Deliveries shape: {df_sales.shape}")
print(df_sales.head())

# Load purchases (for reference)
df_purchases = pl.read_csv(PURCHASES_FILE)
print(f"\nPurchases shape: {df_purchases.shape}")
print(df_purchases.head())

# Load replacement orders
df_replacements = pl.read_csv(REPLACEMENT_FILE)
print(f"\nReplacements shape: {df_replacements.shape}")
print(df_replacements.head())



df_products = pl.read_json(PRODUCT_FILE, infer_schema_length=None)

print(f"\nProduct Data shape: {df_products.shape}")
print(df_products.head())

Sales & Deliveries shape: (7357509, 18)
shape: (5, 18)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ order_num ┆ order_cre ┆ order_cre ┆ requested ┆ … ┆ warehouse ┆ picking_c ┆ picking_c ┆ picking_ │
│ ber       ┆ ated_date ┆ ated_time ┆ _delivery ┆   ┆ _number   ┆ onfirmed_ ┆ onfirmed_ ┆ picked_q │
│ ---       ┆ ---       ┆ ---       ┆ _date     ┆   ┆ ---       ┆ date      ┆ time      ┆ ty       │
│ i64       ┆ str       ┆ i64       ┆ ---       ┆   ┆ i64       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆ str       ┆   ┆           ┆ str       ┆ i64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 10000000  ┆ 2024-09-0 ┆ 336       ┆ 2024-09-0 ┆ … ┆ 3001      ┆ 2024-09-0 ┆ 203837    ┆ 5.0      │
│           ┆ 1         ┆           ┆ 2         ┆   ┆           ┆ 1         ┆           ┆          │
│ 10000000  ┆ 2024-09-0 ┆ 336       

In [5]:
df_sales.columns , df_purchases.columns, df_replacements.columns, df_products.columns

(['order_number',
  'order_created_date',
  'order_created_time',
  'requested_delivery_date',
  'customer_number',
  'order_row_number',
  'product_code',
  'order_qty',
  'sales_unit',
  'delivery_number',
  'plant',
  'storage_location',
  'delivered_qty',
  'transfer_number',
  'warehouse_number',
  'picking_confirmed_date',
  'picking_confirmed_time',
  'picking_picked_qty'],
 ['order_number',
  'po_row_number',
  'customer_number',
  'po_created_date',
  'requested_delivery_date',
  'product_code',
  'plant',
  'storage_location',
  'ordered_qty',
  'unit',
  'received_qty'],
 ['order_number',
  'order_created_date',
  'order_created_time',
  'requested_delivery_date',
  'customer_number',
  'order_row_number',
  'product_code',
  'order_qty',
  'sales_unit',
  'delivery_number',
  'plant',
  'storage_location',
  'delivered_qty',
  'transfer_number',
  'warehouse_number',
  'picking_confirmed_date',
  'picking_confirmed_time',
  'picking_picked_qty'],
 ['salesUnitGtin',
  'sales

In [6]:
def explore_dataframes():
    """Explore the structure and relationships of datasets"""
    
    print("\n" + "="*80)
    print("DATA EXPLORATION")
    print("="*80)
    
    # Sales & Deliveries Analysis
    print("\n--- SALES & DELIVERIES ---")
    print(f"Columns: {df_sales.columns}")
    print(f"\nData types:\n{df_sales.dtypes}")
    print(f"\nNull counts:\n{df_sales.null_count()}")
    print(f"\nUnique values:")
    print(f"  - Orders: {df_sales['order_number'].n_unique()}")
    print(f"  - Customers: {df_sales['customer_number'].n_unique()}")
    print(f"  - Products: {df_sales['product_code'].n_unique()}")
    
    # Check for shortages in sales
    shortage_check = df_sales.with_columns([
        (pl.col('order_qty') > pl.col('delivered_qty')).alias('is_shortage'),
        pl.col('delivered_qty').is_null().alias('is_null_delivery')
    ])
    
    shortage_rate = shortage_check['is_shortage'].mean() * 100
    null_delivery_rate = shortage_check['is_null_delivery'].mean() * 100
    
    print(f"\n  - Shortage rate: {shortage_rate:.2f}%")
    print(f"  - Null delivery rate: {null_delivery_rate:.2f}%")
    
    # Purchases Analysis
    print("\n--- PURCHASES ---")
    print(f"Columns: {df_purchases.columns}")
    print(f"Unique orders: {df_purchases['order_number'].n_unique()}")
    print(f"Unique products: {df_purchases['product_code'].n_unique()}")
    
    # Check for shortages in purchases
    purchase_shortage_check = df_purchases.with_columns([
        (pl.col('ordered_qty') > pl.col('received_qty')).alias('is_shortage')
    ])
    purchase_shortage_rate = purchase_shortage_check['is_shortage'].mean() * 100
    print(f"  - Purchase shortage rate: {purchase_shortage_rate:.2f}%")
    
    # Check overlap between purchases and sales
    purchase_orders = set(df_purchases['order_number'].unique().to_list())
    sales_orders = set(df_sales['order_number'].unique().to_list())
    overlap = len(purchase_orders.intersection(sales_orders))
    print(f"  - Order overlap: {overlap}/{len(purchase_orders)} purchases appear in sales")
    
    # Product Data Analysis
    print("\n--- PRODUCT DATA ---")
    print(f"Columns: {df_products.columns}")
    
    # Check if product_code matches salesUnitGtin or needs mapping
    if 'salesUnitGtin' in df_products.columns:
        print(f"Unique products (salesUnitGtin): {df_products['salesUnitGtin'].n_unique()}")
        
        # Check overlap with sales product_code
        sales_products = set(df_sales['product_code'].unique().to_list())
        product_gtins = set(df_products['salesUnitGtin'].unique().to_list())
        product_overlap = len(sales_products.intersection(product_gtins))
        print(f"  - Product code overlap: {product_overlap}/{len(sales_products)} sales products found in product data")
    
    if 'vendorName' in df_products.columns:
        print(f"\nVendor distribution:")
        print(df_products['vendorName'].value_counts().head(10))
    
    if 'category' in df_products.columns:
        print(f"\nCategory distribution:")
        print(df_products['category'].value_counts().head(10))
    
    # Replacement Orders Analysis
    print("\n--- REPLACEMENT ORDERS ---")
    print(f"Shape: {df_replacements.shape}")
    print(f"Columns: {df_replacements.columns}")
    print("\nNote: Replacement file has same structure as sales. Need to identify replacements differently.")
    print(f"  - Unique orders: {df_replacements['order_number'].n_unique()}")
    print(f"  - Unique products: {df_replacements['product_code'].n_unique()}")
    
    return shortage_check

shortage_check = explore_dataframes()


DATA EXPLORATION

--- SALES & DELIVERIES ---
Columns: ['order_number', 'order_created_date', 'order_created_time', 'requested_delivery_date', 'customer_number', 'order_row_number', 'product_code', 'order_qty', 'sales_unit', 'delivery_number', 'plant', 'storage_location', 'delivered_qty', 'transfer_number', 'warehouse_number', 'picking_confirmed_date', 'picking_confirmed_time', 'picking_picked_qty']

Data types:
[Int64, String, Int64, String, Int64, Int64, Int64, Float64, String, Int64, Int64, Int64, Float64, Int64, Int64, String, Int64, Float64]

Null counts:
shape: (1, 18)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ order_num ┆ order_cre ┆ order_cre ┆ requested ┆ … ┆ warehouse ┆ picking_c ┆ picking_c ┆ picking_ │
│ ber       ┆ ated_date ┆ ated_time ┆ _delivery ┆   ┆ _number   ┆ onfirmed_ ┆ onfirmed_ ┆ picked_q │
│ ---       ┆ ---       ┆ ---       ┆ _date     ┆   ┆ ---       ┆ date      ┆ time      ┆ ty       │
│ u32       ┆ 

In [7]:
def clean_data(df_sales, df_products):
    """Clean and preprocess the data"""
    
    print("\n" + "="*80)
    print("DATA CLEANING")
    print("="*80)
    
    df = df_sales.clone()
    
    # 1. Convert date columns to datetime
    print("\n1. Converting date columns...")
    date_columns = ['order_created_date', 'requested_delivery_date', 'picking_confirmed_date']
    
    for col in date_columns:
        if col in df.columns:
            try:
                df = df.with_columns([
                    pl.col(col).str.strptime(pl.Date, format="%Y-%m-%d", strict=False).alias(col)
                ])
                print(f"  ✓ Converted {col}")
            except Exception as e:
                print(f"  ✗ Error converting {col}: {e}")
    
    # 2. Handle '1970-01-01' dates (placeholder dates)
    print("\n2. Handling placeholder dates...")
    for col in date_columns:
        if col in df.columns and df[col].dtype == pl.Date:
            count_1970 = df.filter(pl.col(col) == datetime(1970, 1, 1).date()).height
            if count_1970 > 0:
                print(f"  - {col}: {count_1970} placeholder dates -> set to null")
                df = df.with_columns([
                    pl.when(pl.col(col) == datetime(1970, 1, 1).date())
                    .then(None)
                    .otherwise(pl.col(col))
                    .alias(col)
                ])
    
    # 3. Handle null delivered_qty (assume 0 if order exists but no delivery)
    print("\n3. Handling null delivered quantities...")
    null_delivered = df['delivered_qty'].is_null().sum()
    print(f"  - Null delivered_qty: {null_delivered}")
    
    df = df.with_columns([
        pl.col('delivered_qty').fill_null(0)
    ])
    
    # 4. Handle null picking_picked_qty
    null_picked = df['picking_picked_qty'].is_null().sum()
    print(f"  - Null picking_picked_qty: {null_picked}")
    df = df.with_columns([
        pl.col('picking_picked_qty').fill_null(0)
    ])
    
    # 5. Create target variable: is_shortage
    print("\n4. Creating target variable...")
    df = df.with_columns([
        (
            (pl.col('order_qty') > pl.col('delivered_qty')) |
            ((pl.col('delivered_qty') == 0) & (pl.col('order_qty') > 0))
        ).cast(pl.Int8).alias('is_shortage')
    ])
    
    shortage_count = df['is_shortage'].sum()
    total_count = df.height
    print(f"  - Shortage instances: {shortage_count}/{total_count} ({shortage_count/total_count*100:.2f}%)")
    
    # 6. Merge with product data (using salesUnitGtin as product_code)
    print("\n5. Merging with product data...")
    print(f"  - Sales product_code dtype: {df['product_code'].dtype}")
    print(f"  - Product salesUnitGtin dtype: {df_products['salesUnitGtin'].dtype}")
    
    original_rows = df.height
    
    # Convert product_code in sales to string to match salesUnitGtin
    df = df.with_columns([
        pl.col('product_code').cast(pl.Utf8).alias('product_code')
    ])
    
    # Rename salesUnitGtin to product_code for joining
    df_products_renamed = df_products.rename({'salesUnitGtin': 'product_code'})
    
    # Ensure product_code in products is also string
    df_products_renamed = df_products_renamed.with_columns([
        pl.col('product_code').cast(pl.Utf8)
    ])
    
    print(f"  - After casting - Sales product_code dtype: {df['product_code'].dtype}")
    print(f"  - After casting - Product product_code dtype: {df_products_renamed['product_code'].dtype}")
    
    df = df.join(df_products_renamed, on='product_code', how='left')
    print(f"  - Rows after merge: {df.height} (original: {original_rows})")
    
    # Check how many products matched
    matched_products = df.filter(pl.col('vendorName').is_not_null()).height
    print(f"  - Rows with product data: {matched_products}/{df.height} ({matched_products/df.height*100:.2f}%)")
    
    # 7. Handle categorical nulls
    print("\n6. Handling categorical nulls...")
    categorical_cols = ['storage_location', 'plant', 'vendorName', 'category', 'temperatureCondition']
    for col in categorical_cols:
        if col in df.columns:
            null_count = df[col].is_null().sum()
            if null_count > 0:
                print(f"  - {col}: {null_count} nulls -> filled with 'UNKNOWN'")
                df = df.with_columns([
                    pl.col(col).fill_null('UNKNOWN')
                ])
    
    # 8. Remove duplicates
    print("\n7. Removing duplicates...")
    original_rows = df.height
    df = df.unique()
    removed = original_rows - df.height
    if removed > 0:
        print(f"  - Removed {removed} duplicates")
    else:
        print(f"  - No duplicates found")
    
    print("\n✓ Cleaning complete!")
    print(f"Final shape: {df.shape}")
    
    return df

# Clean the data
df_clean = clean_data(df_sales, df_products)
print("\nCleaned data sample:")
print(df_clean.select(['order_number', 'product_code', 'order_qty', 'delivered_qty', 'is_shortage', 'vendorName', 'category']).head(10))


DATA CLEANING

1. Converting date columns...
  ✓ Converted order_created_date
  ✓ Converted requested_delivery_date
  ✓ Converted picking_confirmed_date

2. Handling placeholder dates...
  - picking_confirmed_date: 360459 placeholder dates -> set to null

3. Handling null delivered quantities...
  - Null delivered_qty: 147370
  - Null picking_picked_qty: 173392

4. Creating target variable...
  - Shortage instances: 292511/7357509 (3.98%)

5. Merging with product data...
  - Sales product_code dtype: Int64
  - Product salesUnitGtin dtype: String
  - After casting - Sales product_code dtype: String
  - After casting - Product product_code dtype: String
  - Rows after merge: 7357509 (original: 7357509)
  - Rows with product data: 0/7357509 (0.00%)

6. Handling categorical nulls...
  - storage_location: 147384 nulls -> filled with 'UNKNOWN'
  - plant: 147370 nulls -> filled with 'UNKNOWN'
  - vendorName: 7357509 nulls -> filled with 'UNKNOWN'
  - category: 7357509 nulls -> filled with 'U

In [8]:
def create_date_features(df):
    """Create time-based features"""
    
    print("\n--- Creating Date Features ---")
    
    # From order_created_date
    df = df.with_columns([
        pl.col('order_created_date').dt.weekday().alias('order_day_of_week'),
        pl.col('order_created_date').dt.day().alias('order_day_of_month'),
        pl.col('order_created_date').dt.month().alias('order_month'),
        pl.col('order_created_date').dt.quarter().alias('order_quarter'),
        pl.col('order_created_date').dt.year().alias('order_year'),
        (pl.col('order_created_date').dt.weekday() >= 5).cast(pl.Int8).alias('is_weekend')
    ])
    print("  ✓ Created day_of_week, day_of_month, month, quarter, year, is_weekend")
    
    # Calculate lead time
    df = df.with_columns([
        (pl.col('requested_delivery_date') - pl.col('order_created_date'))
        .dt.total_days()
        .alias('lead_time_days')
    ])
    print("  ✓ Created lead_time_days")
    
    # Time of day from order_created_time
    try:
        df = df.with_columns([
            pl.col('order_created_time').str.slice(0, 2).cast(pl.Int32, strict=False).alias('order_hour')
        ])
        print("  ✓ Created order_hour")
    except Exception as e:
        print(f"  ✗ Could not extract order_hour: {e}")
    
    return df


def create_product_features(df):
    """Create product-level aggregated features"""
    
    print("\n--- Creating Product Features ---")
    
    # Sort by date for proper temporal aggregation
    df = df.sort('order_created_date')
    
    # 1. Historical shortage rate per product
    product_stats = df.group_by('product_code').agg([
        pl.col('is_shortage').mean().alias('product_shortage_rate'),
        pl.col('order_qty').mean().alias('product_avg_order_qty'),
        pl.col('order_qty').std().alias('product_order_qty_std'),
        pl.col('delivered_qty').mean().alias('product_avg_delivered_qty'),
        pl.col('order_number').n_unique().alias('product_order_count')
    ])
    
    df = df.join(product_stats, on='product_code', how='left', suffix='_stat')
    print("  ✓ Created product_shortage_rate, product_avg_order_qty, product_order_qty_std")
    
    # 2. Category-based features
    if 'category' in df.columns:
        category_stats = df.group_by('category').agg([
            pl.col('is_shortage').mean().alias('category_shortage_rate')
        ])
        df = df.join(category_stats, on='category', how='left', suffix='_cat')
        print("  ✓ Created category_shortage_rate")
    
    # 3. Temperature condition features (perishables might have higher shortage)
    if 'temperatureCondition' in df.columns:
        temp_stats = df.group_by('temperatureCondition').agg([
            pl.col('is_shortage').mean().alias('temp_condition_shortage_rate')
        ])
        df = df.join(temp_stats, on='temperatureCondition', how='left', suffix='_temp')
        print("  ✓ Created temp_condition_shortage_rate")
    
    return df


def create_customer_features(df):
    """Create customer-level aggregated features"""
    
    print("\n--- Creating Customer Features ---")
    
    # Customer statistics
    customer_stats = df.group_by('customer_number').agg([
        pl.col('is_shortage').mean().alias('customer_shortage_rate'),
        pl.col('order_qty').sum().alias('customer_total_order_qty'),
        pl.col('order_number').n_unique().alias('customer_order_count'),
        pl.col('lead_time_days').mean().alias('customer_avg_lead_time')
    ])
    
    df = df.join(customer_stats, on='customer_number', how='left', suffix='_cust')
    print("  ✓ Created customer_shortage_rate, customer_total_order_qty, customer_order_count")
    
    # Customer-Product combination
    customer_product_stats = df.group_by(['customer_number', 'product_code']).agg([
        pl.col('is_shortage').mean().alias('customer_product_shortage_rate'),
        pl.col('order_number').n_unique().alias('customer_product_order_count')
    ])
    
    df = df.join(customer_product_stats, on=['customer_number', 'product_code'], how='left', suffix='_cp')
    print("  ✓ Created customer_product_shortage_rate")
    
    return df


def create_vendor_features(df):
    """Create vendor/supplier reliability features"""
    
    print("\n--- Creating Vendor Features ---")
    
    if 'vendorName' in df.columns:
        vendor_stats = df.group_by('vendorName').agg([
            pl.col('is_shortage').mean().alias('vendor_shortage_rate'),
            pl.col('order_number').n_unique().alias('vendor_order_count')
        ])
        
        df = df.join(vendor_stats, on='vendorName', how='left', suffix='_vend')
        print("  ✓ Created vendor_shortage_rate (vendor reliability)")
    
    return df


def create_location_features(df):
    """Create plant and storage location features"""
    
    print("\n--- Creating Location Features ---")
    
    # Plant statistics
    plant_stats = df.group_by('plant').agg([
        pl.col('is_shortage').mean().alias('plant_shortage_rate'),
        pl.col('order_number').n_unique().alias('plant_order_count')
    ])
    df = df.join(plant_stats, on='plant', how='left', suffix='_plant')
    print("  ✓ Created plant_shortage_rate")
    
    # Storage location statistics
    storage_stats = df.group_by('storage_location').agg([
        pl.col('is_shortage').mean().alias('storage_shortage_rate')
    ])
    df = df.join(storage_stats, on='storage_location', how='left', suffix='_stor')
    print("  ✓ Created storage_shortage_rate")
    
    return df


def create_replacement_features(df, df_replacements):
    """Create features indicating products that often need replacements"""
    
    print("\n--- Creating Replacement Features ---")
    
    # Since replacements has same structure as sales, we need to identify which orders are replacements
    # For now, count how often each product appears in replacement file
    replacement_counts = df_replacements.group_by('product_code').agg([
        pl.count().alias('replacement_frequency')
    ])
    
    # Cast product_code to string in replacement_counts to match
    replacement_counts = replacement_counts.with_columns([
        pl.col('product_code').cast(pl.Utf8)
    ])
    
    df = df.join(replacement_counts, on='product_code', how='left')
    df = df.with_columns([
        pl.col('replacement_frequency').fill_null(0)
    ])
    
    print("  ✓ Created replacement_frequency")
    
    return df


# Apply all feature engineering
print("\n" + "="*80)
print("FEATURE ENGINEERING")
print("="*80)

df_features = df_clean.clone()
df_features = create_date_features(df_features)
df_features = create_product_features(df_features)
df_features = create_customer_features(df_features)
df_features = create_vendor_features(df_features)
df_features = create_location_features(df_features)
df_features = create_replacement_features(df_features, df_replacements)

print("\n✓ Feature engineering complete!")
print(f"Total columns: {len(df_features.columns)}")
print(f"Shape: {df_features.shape}")

# Show a sample of features
print("\nSample of engineered features:")
feature_sample_cols = ['order_number', 'product_code', 'is_shortage', 'product_shortage_rate', 
                       'customer_shortage_rate', 'vendor_shortage_rate', 'lead_time_days']
available_cols = [col for col in feature_sample_cols if col in df_features.columns]
print(df_features.select(available_cols).head(10))


FEATURE ENGINEERING

--- Creating Date Features ---
  ✓ Created day_of_week, day_of_month, month, quarter, year, is_weekend
  ✓ Created lead_time_days
  ✗ Could not extract order_hour: invalid series dtype: expected `String`, got `i64` for series with name `order_created_time`

--- Creating Product Features ---
  ✓ Created product_shortage_rate, product_avg_order_qty, product_order_qty_std
  ✓ Created category_shortage_rate
  ✓ Created temp_condition_shortage_rate

--- Creating Customer Features ---
  ✓ Created customer_shortage_rate, customer_total_order_qty, customer_order_count
  ✓ Created customer_product_shortage_rate

--- Creating Vendor Features ---
  ✓ Created vendor_shortage_rate (vendor reliability)

--- Creating Location Features ---
  ✓ Created plant_shortage_rate
  ✓ Created storage_shortage_rate

--- Creating Replacement Features ---
  ✓ Created replacement_frequency

✓ Feature engineering complete!
Total columns: 57
Shape: (7350226, 57)

Sample of engineered features:
s

C:\Users\mfrfo\AppData\Local\Temp\ipykernel_77616\91580802.py:152: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('replacement_frequency')


In [9]:
# Let's investigate the product code mismatch
print("Investigating product code mismatch...")
print("\nSample product codes from sales:")
print(df_sales.select('product_code').unique().head(20))

print("\nSample salesUnitGtin from products:")
print(df_products.select('salesUnitGtin').unique().head(20))

# Check if salesUnitGtin might need leading zeros or different formatting
print("\nSalesUnitGtin data types and samples:")
print(df_products.select(['salesUnitGtin']).head(20))

Investigating product code mismatch...

Sample product codes from sales:
shape: (20, 1)
┌──────────────┐
│ product_code │
│ ---          │
│ i64          │
╞══════════════╡
│ 415793       │
│ 406355       │
│ 400722       │
│ 402437       │
│ 405834       │
│ 403351       │
│ 407939       │
│ 410696       │
│ 410949       │
│ 401764       │
│ 406611       │
│ 405444       │
│ 405316       │
│ 400987       │
│ 409892       │
│ 407814       │
│ 406873       │
│ 407016       │
│ 400996       │
│ 413575       │
└──────────────┘

Sample salesUnitGtin from products:
shape: (20, 1)
┌───────────────┐
│ salesUnitGtin │
│ ---           │
│ str           │
╞═══════════════╡
│ 6413467612305 │
│ 8591346304553 │
│ 7391440885009 │
│ 8414933008583 │
│ 6408431305109 │
│ 6411440060600 │
│ 2300760200000 │
│ 6413466101503 │
│ 6430018362052 │
│ 6408430339457 │
│ 6418047431512 │
│ 7310800017690 │
│ 7622200609680 │
│ 6409100272616 │
│ 6418675197170 │
│ 6410381083068 │
│ 6430025257457 │
│ 5703538134777 │
│ 73

In [10]:
# Let's examine the full structure of product data to find a mapping
print("Full product data structure:")
print(df_products.columns)
print("\nSample product record:")
print(df_products.head(1))

# Check if there's a nested structure that might contain product_code
print("\n--- Checking nested fields ---")

# Check 'units' field
if 'units' in df_products.columns:
    print("\nSample 'units' field:")
    sample_units = df_products['units'][0]
    print(type(sample_units))
    print(sample_units)

# Check 'synkkaData' field
if 'synkkaData' in df_products.columns:
    print("\nSample 'synkkaData' field:")
    sample_synkka = df_products['synkkaData'][0]
    print(type(sample_synkka))
    print(sample_synkka)

# Check 'substitutions' field
if 'substitutions' in df_products.columns:
    print("\nSample 'substitutions' field:")
    sample_sub = df_products['substitutions'][0]
    print(type(sample_sub))
    print(sample_sub)

Full product data structure:
['salesUnitGtin', 'salesUnit', 'baseUnit', 'category', 'allowedLotSize', 'deleted', 'temperatureCondition', 'vendorName', 'countryOfOrigin', 'synkkaData', 'units', 'deposits', 'substitutions']

Sample product record:
shape: (1, 13)
┌────────────┬───────────┬──────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ salesUnitG ┆ salesUnit ┆ baseUnit ┆ category ┆ … ┆ synkkaDat ┆ units     ┆ deposits  ┆ substitut │
│ tin        ┆ ---       ┆ ---      ┆ ---      ┆   ┆ a         ┆ ---       ┆ ---       ┆ ions      │
│ ---        ┆ str       ┆ str      ┆ str      ┆   ┆ ---       ┆ list[stru ┆ list[stru ┆ ---       │
│ str        ┆           ┆          ┆          ┆   ┆ struct[26 ┆ ct[3]]    ┆ ct[3]]    ┆ list[stru │
│            ┆           ┆          ┆          ┆   ┆ ]         ┆           ┆           ┆ ct[4]]    │
╞════════════╪═══════════╪══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 6409460002 ┆ RAS       ┆ RAS  

In [11]:
import pandas as pd

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import time

# Fix order_hour extraction
print("\n" + "="*80)
print("FIXING ORDER HOUR EXTRACTION")
print("="*80)

df_features = df_features.with_columns([
    (pl.col('order_created_time') // 10000).alias('order_hour')
])
print("✓ Added order_hour feature")
print(f"Sample order_hour values: {df_features['order_hour'].head(10).to_list()}")

# Prepare data for modeling
def prepare_modeling_data(df):
    """Prepare data for machine learning"""
    
    print("\n" + "="*80)
    print("PREPARING DATA FOR MODELING")
    print("="*80)
    
    # Select features for modeling
    feature_cols = [
        # Quantity features
        'order_qty',
        'picking_picked_qty',
        
        # Date features
        'order_day_of_week',
        'order_day_of_month',
        'order_month',
        'order_quarter',
        'is_weekend',
        'lead_time_days',
        'order_hour',
        
        # Product features
        'product_shortage_rate',
        'product_avg_order_qty',
        'product_order_qty_std',
        'product_order_count',
        
        # Customer features
        'customer_shortage_rate',
        'customer_total_order_qty',
        'customer_order_count',
        'customer_avg_lead_time',
        
        # Customer-Product features
        'customer_product_shortage_rate',
        'customer_product_order_count',
        
        # Vendor features (all UNKNOWN but keeping structure)
        'vendor_shortage_rate',
        'vendor_order_count',
        
        # Location features
        'plant_shortage_rate',
        'storage_shortage_rate',
        
        # Replacement features
        'replacement_frequency',
        
        # Category features
        'category_shortage_rate',
        'temp_condition_shortage_rate'
    ]
    
    # Filter to only columns that exist
    feature_cols = [col for col in feature_cols if col in df.columns]
    
    print(f"\nSelected {len(feature_cols)} features:")
    for i, col in enumerate(feature_cols, 1):
        print(f"  {i:2d}. {col}")
    
    # Add target and metadata
    cols_to_keep = feature_cols + ['is_shortage', 'order_created_date', 'order_number', 'product_code', 'customer_number']
    cols_to_keep = [col for col in cols_to_keep if col in df.columns]
    
    df_model = df.select(cols_to_keep)
    
    # Remove rows with nulls in features
    print(f"\nRows before null removal: {df_model.height:,}")
    df_model = df_model.drop_nulls(subset=feature_cols)
    print(f"Rows after null removal: {df_model.height:,}")
    
    # Check for any remaining nulls
    null_counts = df_model.select(feature_cols).null_count()
    total_nulls = sum(null_counts.row(0))
    
    if total_nulls > 0:
        print(f"\n⚠ Warning: {total_nulls} nulls remain")
    else:
        print("\n✓ No nulls in features")
    
    # Check class balance
    print("\n--- Class Balance ---")
    shortage_rate = df_model['is_shortage'].mean()
    print(f"Shortage rate: {shortage_rate*100:.2f}%")
    print(f"Non-shortage rate: {(1-shortage_rate)*100:.2f}%")
    print(f"Class imbalance ratio: {(1-shortage_rate)/shortage_rate:.1f}:1")
    
    # Time-based split
    print("\n--- Creating Time-Based Train/Test Split ---")
    df_model = df_model.sort('order_created_date')
    
    # Use 80% for training, 20% for testing
    split_idx = int(df_model.height * 0.8)
    
    df_train = df_model[:split_idx]
    df_test = df_model[split_idx:]
    
    print(f"\nTraining set: {df_train.height:,} rows")
    print(f"  Date range: {df_train['order_created_date'].min()} to {df_train['order_created_date'].max()}")
    print(f"  Shortage rate: {df_train['is_shortage'].mean()*100:.2f}%")
    
    print(f"\nTest set: {df_test.height:,} rows")
    print(f"  Date range: {df_test['order_created_date'].min()} to {df_test['order_created_date'].max()}")
    print(f"  Shortage rate: {df_test['is_shortage'].mean()*100:.2f}%")
    
    # Convert to numpy for sklearn
    X_train = df_train.select(feature_cols).to_numpy()
    y_train = df_train.select('is_shortage').to_numpy().ravel()
    
    X_test = df_test.select(feature_cols).to_numpy()
    y_test = df_test.select('is_shortage').to_numpy().ravel()
    
    print(f"\n✓ Data prepared for modeling!")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test, feature_cols, df_train, df_test

X_train, X_test, y_train, y_test, feature_cols, df_train, df_test = prepare_modeling_data(df_features)


# Evaluation function
def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """Evaluate model performance"""
    
    print(f"\n{'='*80}")
    print(f"{model_name} PERFORMANCE")
    print(f"{'='*80}")
    
    accuracy = (y_true == y_pred).mean()
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    print(f"\nMetrics:")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f} (% of predicted shortages that were correct)")
    print(f"  Recall:    {recall:.4f} (% of actual shortages we caught)")
    print(f"  F1-Score:  {f1:.4f} (harmonic mean)")
    
    try:
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        print(f"  ROC AUC:   {roc_auc:.4f}")
    except:
        roc_auc = None
        print(f"  ROC AUC:   N/A")
    
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nConfusion Matrix:")
    print(f"{'':20} Predicted: No | Predicted: Yes")
    print(f"Actual: No Shortage  {cm[0,0]:>10,}   |   {cm[0,1]:>10,}")
    print(f"Actual: Shortage     {cm[1,0]:>10,}   |   {cm[1,1]:>10,}")
    
    total_actual = cm[1,0] + cm[1,1]
    total_predicted = cm[0,1] + cm[1,1]
    
    print(f"\nBusiness Impact:")
    print(f"  • Actual shortages: {total_actual:,}")
    print(f"  • Predicted shortages: {total_predicted:,}")
    print(f"  • Caught {cm[1,1]:,} shortages ({cm[1,1]/total_actual*100:.1f}%)")
    print(f"  • Missed {cm[1,0]:,} shortages ({cm[1,0]/total_actual*100:.1f}%)")
    print(f"  • False alarms: {cm[0,1]:,}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'confusion_matrix': cm
    }


# MODEL 1: Logistic Regression
print("\n" + "="*80)
print("MODEL 1: LOGISTIC REGRESSION (Baseline)")
print("="*80)

start = time.time()
lr_model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced', n_jobs=-1)
lr_model.fit(X_train, y_train)
print(f"Training time: {time.time()-start:.1f}s")

y_pred_lr = lr_model.predict(X_test)
y_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]
lr_metrics = evaluate_model(y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression")


# MODEL 2: Random Forest
print("\n" + "="*80)
print("MODEL 2: RANDOM FOREST")
print("="*80)

start = time.time()
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=100,
    min_samples_leaf=50,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
print(f"Training time: {time.time()-start:.1f}s")

y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]
rf_metrics = evaluate_model(y_test, y_pred_rf, y_pred_proba_rf, "Random Forest")

print("\n🌟 Top 15 Most Important Features:")
feat_imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)
for i, row in feat_imp.head(15).iterrows():
    print(f"  {row['feature']:.<45} {row['importance']:.4f}")


# MODEL 3: XGBoost
print("\n" + "="*80)
print("MODEL 3: XGBOOST (Advanced)")
print("="*80)

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.1f}")

start = time.time()
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)
print(f"Training time: {time.time()-start:.1f}s")

y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
xgb_metrics = evaluate_model(y_test, y_pred_xgb, y_pred_proba_xgb, "XGBoost")

print("\n🌟 Top 15 Most Important Features (XGBoost):")
feat_imp_xgb = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)
for i, row in feat_imp_xgb.head(15).iterrows():
    print(f"  {row['feature']:.<45} {row['importance']:.4f}")


# COMPARISON
print("\n" + "="*80)
print("📊 MODEL COMPARISON")
print("="*80)

comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [lr_metrics['accuracy'], rf_metrics['accuracy'], xgb_metrics['accuracy']],
    'Precision': [lr_metrics['precision'], rf_metrics['precision'], xgb_metrics['precision']],
    'Recall': [lr_metrics['recall'], rf_metrics['recall'], xgb_metrics['recall']],
    'F1': [lr_metrics['f1'], rf_metrics['f1'], xgb_metrics['f1']],
    'ROC-AUC': [lr_metrics.get('roc_auc', 0), rf_metrics.get('roc_auc', 0), xgb_metrics.get('roc_auc', 0)]
})
print("\n" + comparison.to_string(index=False))

# Save best model
import pickle
print("\n" + "="*80)
print("💾 SAVING MODEL")
print("="*80)

best_model = xgb_model
artifacts = {
    'model': best_model,
    'feature_cols': feature_cols,
    'feature_importance': feat_imp_xgb,
    'threshold': 0.5  # Can be tuned later
}

with open('shortage_predictor_model.pkl', 'wb') as f:
    pickle.dump(artifacts, f)
print("✓ Saved: shortage_predictor_model.pkl")

# Save processed test data
df_test_with_pred = df_test.with_columns([
    pl.lit(y_pred_xgb).alias('predicted_shortage'),
    pl.lit(y_pred_proba_xgb).alias('shortage_probability')
])
df_test_with_pred.write_parquet('test_predictions.parquet')
print("✓ Saved: test_predictions.parquet")

print("\n✅ Modeling complete!")


FIXING ORDER HOUR EXTRACTION
✓ Added order_hour feature
Sample order_hour values: [16, 15, 20, 13, 14, 12, 23, 9, 13, 16]

PREPARING DATA FOR MODELING

Selected 26 features:
   1. order_qty
   2. picking_picked_qty
   3. order_day_of_week
   4. order_day_of_month
   5. order_month
   6. order_quarter
   7. is_weekend
   8. lead_time_days
   9. order_hour
  10. product_shortage_rate
  11. product_avg_order_qty
  12. product_order_qty_std
  13. product_order_count
  14. customer_shortage_rate
  15. customer_total_order_qty
  16. customer_order_count
  17. customer_avg_lead_time
  18. customer_product_shortage_rate
  19. customer_product_order_count
  20. vendor_shortage_rate
  21. vendor_order_count
  22. plant_shortage_rate
  23. storage_shortage_rate
  24. replacement_frequency
  25. category_shortage_rate
  26. temp_condition_shortage_rate

Rows before null removal: 7,350,226
Rows after null removal: 7,349,594

✓ No nulls in features

--- Class Balance ---
Shortage rate: 3.98%
Non-sh

In [13]:
import pickle
import polars as pl
import numpy as np
from datetime import datetime

class ShortagePredictor:
    """Pipeline for predicting order shortages"""
    
    def __init__(self, model_path='shortage_predictor_model.pkl'):
        """Load the trained model"""
        print("Loading model...")
        with open(model_path, 'rb') as f:
            artifacts = pickle.load(f)
        
        self.model = artifacts['model']
        self.feature_cols = artifacts['feature_cols']
        self.feature_importance = artifacts['feature_importance']
        self.threshold = artifacts.get('threshold', 0.5)
        
        print(f"✓ Model loaded with {len(self.feature_cols)} features")
        
        # Store historical aggregations for feature engineering
        self.historical_stats = None
    
    def load_historical_data(self, df_features):
        """
        Load and store historical statistics for feature engineering
        This should be called once with your historical data
        """
        print("\nLoading historical statistics...")
        
        # Product stats
        self.product_stats = df_features.group_by('product_code').agg([
            pl.col('is_shortage').mean().alias('product_shortage_rate'),
            pl.col('order_qty').mean().alias('product_avg_order_qty'),
            pl.col('order_qty').std().alias('product_order_qty_std'),
            pl.col('order_number').n_unique().alias('product_order_count')
        ])
        
        # Customer stats
        self.customer_stats = df_features.group_by('customer_number').agg([
            pl.col('is_shortage').mean().alias('customer_shortage_rate'),
            pl.col('order_qty').sum().alias('customer_total_order_qty'),
            pl.col('order_number').n_unique().alias('customer_order_count'),
            pl.col('lead_time_days').mean().alias('customer_avg_lead_time')
        ])
        
        # Customer-Product stats
        self.customer_product_stats = df_features.group_by(['customer_number', 'product_code']).agg([
            pl.col('is_shortage').mean().alias('customer_product_shortage_rate'),
            pl.col('order_number').n_unique().alias('customer_product_order_count')
        ])
        
        # Location stats
        self.plant_stats = df_features.group_by('plant').agg([
            pl.col('is_shortage').mean().alias('plant_shortage_rate')
        ])
        
        self.storage_stats = df_features.group_by('storage_location').agg([
            pl.col('is_shortage').mean().alias('storage_shortage_rate')
        ])
        
        # Category stats (all UNKNOWN but keep for structure)
        self.category_stats = df_features.group_by('category').agg([
            pl.col('is_shortage').mean().alias('category_shortage_rate')
        ])
        
        self.temp_stats = df_features.group_by('temperatureCondition').agg([
            pl.col('is_shortage').mean().alias('temp_condition_shortage_rate')
        ])
        
        # Vendor stats
        self.vendor_stats = df_features.group_by('vendorName').agg([
            pl.col('is_shortage').mean().alias('vendor_shortage_rate'),
            pl.col('order_number').n_unique().alias('vendor_order_count')
        ])
        
        # Replacement stats
        self.replacement_stats = df_features.group_by('product_code').agg([
            pl.count().alias('replacement_frequency')
        ])
        
        print("✓ Historical statistics loaded")
    
    def engineer_features(self, df_new_orders):
        """
        Engineer features for new orders
        df_new_orders should have columns: order_number, order_created_date, order_created_time,
        requested_delivery_date, customer_number, product_code, order_qty, plant, storage_location, etc.
        """
        df = df_new_orders.clone()
        
        # Date features
        df = df.with_columns([
            pl.col('order_created_date').dt.weekday().alias('order_day_of_week'),
            pl.col('order_created_date').dt.day().alias('order_day_of_month'),
            pl.col('order_created_date').dt.month().alias('order_month'),
            pl.col('order_created_date').dt.quarter().alias('order_quarter'),
            (pl.col('order_created_date').dt.weekday() >= 5).cast(pl.Int8).alias('is_weekend'),
            ((pl.col('requested_delivery_date') - pl.col('order_created_date')).dt.total_days()).alias('lead_time_days'),
            (pl.col('order_created_time') // 10000).alias('order_hour')
        ])
        
        # Add picking_picked_qty (for new orders, we don't have this yet, so use 0 or order_qty as estimate)
        if 'picking_picked_qty' not in df.columns:
            df = df.with_columns([
                pl.lit(0).alias('picking_picked_qty')  # Default to 0 for new orders
            ])
        
        # Join with historical stats
        df = df.join(self.product_stats, on='product_code', how='left')
        df = df.join(self.customer_stats, on='customer_number', how='left')
        df = df.join(self.customer_product_stats, on=['customer_number', 'product_code'], how='left')
        df = df.join(self.plant_stats, on='plant', how='left')
        df = df.join(self.storage_stats, on='storage_location', how='left')
        
        # Handle nulls (new customers/products)
        df = df.with_columns([
            pl.col('product_shortage_rate').fill_null(0.04),  # Use global average
            pl.col('product_avg_order_qty').fill_null(pl.col('order_qty')),
            pl.col('product_order_qty_std').fill_null(0),
            pl.col('product_order_count').fill_null(0),
            pl.col('customer_shortage_rate').fill_null(0.04),
            pl.col('customer_total_order_qty').fill_null(0),
            pl.col('customer_order_count').fill_null(0),
            pl.col('customer_avg_lead_time').fill_null(pl.col('lead_time_days')),
            pl.col('customer_product_shortage_rate').fill_null(0.04),
            pl.col('customer_product_order_count').fill_null(0),
            pl.col('plant_shortage_rate').fill_null(0.04),
            pl.col('storage_shortage_rate').fill_null(0.04)
        ])
        
        # Add remaining features with defaults
        if 'vendorName' in df.columns:
            df = df.join(self.vendor_stats, on='vendorName', how='left')
        
        df = df.with_columns([
            pl.lit(0.04).alias('vendor_shortage_rate') if 'vendor_shortage_rate' not in df.columns else pl.col('vendor_shortage_rate').fill_null(0.04),
            pl.lit(0).alias('vendor_order_count') if 'vendor_order_count' not in df.columns else pl.col('vendor_order_count').fill_null(0),
            pl.lit(0).alias('replacement_frequency') if 'replacement_frequency' not in df.columns else pl.col('replacement_frequency').fill_null(0),
            pl.lit(0.04).alias('category_shortage_rate') if 'category_shortage_rate' not in df.columns else pl.col('category_shortage_rate').fill_null(0.04),
            pl.lit(0.04).alias('temp_condition_shortage_rate') if 'temp_condition_shortage_rate' not in df.columns else pl.col('temp_condition_shortage_rate').fill_null(0.04)
        ])
        
        return df
    
    def predict(self, df_new_orders):
        """
        Predict shortages for new orders
        Returns DataFrame with predictions and probabilities
        """
        print(f"\nPredicting shortages for {df_new_orders.height} orders...")
        
        # Engineer features
        df_features = self.engineer_features(df_new_orders)
        
        # Select only the features used in training
        X = df_features.select(self.feature_cols).to_numpy()
        
        # Predict
        predictions = self.model.predict(X)
        probabilities = self.model.predict_proba(X)[:, 1]
        
        # Add predictions to dataframe
        df_result = df_features.with_columns([
            pl.lit(predictions).alias('predicted_shortage'),
            pl.lit(probabilities).alias('shortage_probability')
        ])
        
        shortage_count = predictions.sum()
        print(f"✓ Predicted {shortage_count} shortages ({shortage_count/len(predictions)*100:.1f}%)")
        
        return df_result
    
    def get_high_risk_orders(self, df_predictions, threshold=0.7):
        """Get orders with high shortage risk"""
        high_risk = df_predictions.filter(
            pl.col('shortage_probability') >= threshold
        ).sort('shortage_probability', descending=True)
        
        return high_risk
    
    def explain_prediction(self, df_row):
        """Explain why an order was predicted as shortage"""
        features = df_row.select(self.feature_cols).to_numpy()[0]
        
        print("\n" + "="*60)
        print("SHORTAGE RISK EXPLANATION")
        print("="*60)
        print(f"\nOrder: {df_row['order_number'][0]}")
        print(f"Product: {df_row['product_code'][0]}")
        print(f"Customer: {df_row['customer_number'][0]}")
        print(f"Shortage Probability: {df_row['shortage_probability'][0]:.1%}")
        
        print("\nTop Risk Factors:")
        # Get feature contributions
        feature_values = {}
        for i, col in enumerate(self.feature_cols):
            importance = self.feature_importance[
                self.feature_importance['feature'] == col
            ]['importance'].values[0] if col in self.feature_importance['feature'].values else 0
            
            feature_values[col] = {
                'value': features[i],
                'importance': importance,
                'contribution': features[i] * importance
            }
        
        # Sort by contribution
        sorted_features = sorted(feature_values.items(), 
                                key=lambda x: abs(x[1]['contribution']), 
                                reverse=True)
        
        for i, (feat, info) in enumerate(sorted_features[:10], 1):
            print(f"  {i:2d}. {feat:.<40} {info['value']:.4f}")

# Initialize predictor
predictor = ShortagePredictor('shortage_predictor_model.pkl')

# Load historical data for feature engineering
predictor.load_historical_data(df_features)

print("\n✅ Predictor ready!")

Loading model...
✓ Model loaded with 26 features

Loading historical statistics...


C:\Users\mfrfo\AppData\Local\Temp\ipykernel_77616\2976920318.py:80: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('replacement_frequency')


✓ Historical statistics loaded

✅ Predictor ready!


In [ ]:
# Step 2: Predict on New Orders (Fixed)

print("\n" + "="*80)
print("EXAMPLE: PREDICTING ON NEW ORDERS")
print("="*80)

# Get the columns available in df_test
print("Available columns in df_test:")
print(df_test.columns)

# We need to use the original df_features to get all columns
# Filter to the test date range
test_start_date = df_test['order_created_date'].min()
test_end_date = df_test['order_created_date'].max()

print(f"\nTest date range: {test_start_date} to {test_end_date}")

# Get test orders from original df_features
df_test_orders_full = df_features.filter(
    (pl.col('order_created_date') >= test_start_date) &
    (pl.col('order_created_date') <= test_end_date)
).head(1000)  # Sample 1000 orders

# Select the columns we need for prediction
required_cols = [
    'order_number', 'order_created_date', 'order_created_time', 
    'requested_delivery_date', 'customer_number', 'product_code', 
    'order_qty', 'plant', 'storage_location', 'picking_picked_qty'
]

# Check which columns exist
available_cols = [col for col in required_cols if col in df_test_orders_full.columns]
print(f"\nAvailable columns: {available_cols}")

df_test_orders = df_test_orders_full.select(available_cols)

print(f"\nSample of test orders:")
print(df_test_orders.head())

# Predict
predictions = predictor.predict(df_test_orders)

# Show results
print("\n📊 Sample Predictions:")
result_cols = ['order_number', 'product_code', 'customer_number', 'order_qty',
               'predicted_shortage', 'shortage_probability']
available_result_cols = [col for col in result_cols if col in predictions.columns]

print(predictions.select(available_result_cols).sort('shortage_probability', descending=True).head(10))

# Get high-risk orders
high_risk = predictor.get_high_risk_orders(predictions, threshold=0.8)
print(f"\n⚠️  Found {high_risk.height} high-risk orders (≥80% shortage probability)")

if high_risk.height > 0:
    print(high_risk.select(available_result_cols).head(10))
    
    # Explain a specific prediction
    print("\n" + "="*80)
    print("DETAILED EXPLANATION FOR HIGHEST RISK ORDER")
    print("="*80)
    predictor.explain_prediction(high_risk.head(1))


EXAMPLE: PREDICTING ON NEW ORDERS
Available columns in df_test:
['order_qty', 'picking_picked_qty', 'order_day_of_week', 'order_day_of_month', 'order_month', 'order_quarter', 'is_weekend', 'lead_time_days', 'order_hour', 'product_shortage_rate', 'product_avg_order_qty', 'product_order_qty_std', 'product_order_count', 'customer_shortage_rate', 'customer_total_order_qty', 'customer_order_count', 'customer_avg_lead_time', 'customer_product_shortage_rate', 'customer_product_order_count', 'vendor_shortage_rate', 'vendor_order_count', 'plant_shortage_rate', 'storage_shortage_rate', 'replacement_frequency', 'category_shortage_rate', 'temp_condition_shortage_rate', 'is_shortage', 'order_created_date', 'order_number', 'product_code', 'customer_number']

Test date range: 2025-07-22 to 2025-09-30

Available columns: ['order_number', 'order_created_date', 'order_created_time', 'requested_delivery_date', 'customer_number', 'product_code', 'order_qty', 'plant', 'storage_location', 'picking_picked_q